In [0]:
%sql
CREATE CATALOG IF NOT EXISTS yogurt;
CREATE SCHEMA IF NOT EXISTS yogurt.bronze;
CREATE SCHEMA IF NOT EXISTS yogurt.silver;
CREATE SCHEMA IF NOT EXISTS yogurt.monitoring;


In [0]:
%sql
USE CATALOG yogurt;
USE SCHEMA silver;

SHOW TABLES IN yogurt.bronze;



In [0]:
%sql
SHOW TABLES IN yogurt.silver;


In [0]:
%sql
SELECT SUM(li.l_quantity) AS total_items
FROM yogurt.silver.orders o
JOIN yogurt.silver.lineitem li ON li.l_orderkey = o.o_orderkey
WHERE o.o_custkey = 446044;


In [0]:
%sql
SELECT MIN(o_orderdate) AS first_purchase_date
FROM yogurt.silver.orders
WHERE o_custkey = 564787;

In [0]:
%sql
SELECT COUNT(*) AS total_orders
FROM yogurt.silver.orders
WHERE o_custkey = 85909;


In [0]:
%sql
WITH c_orders AS (
  SELECT o.o_custkey, COUNT(*) AS cnt
  FROM yogurt.silver.orders o
  GROUP BY o.o_custkey
),
max_c AS (
  SELECT o.o_custkey
  FROM c_orders o
  ORDER BY cnt DESC
  LIMIT 1
)
SELECT n.n_name AS nationality
FROM yogurt.silver.customer c
JOIN yogurt.silver.nation n ON c.c_nationkey = n.n_nationkey
JOIN max_c ON max_c.o_custkey = c.c_custkey;


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS yogurt.monitoring;

CREATE OR REPLACE TABLE yogurt.monitoring.b4_answers AS
SELECT 'Q1_total_items_446044' AS q,
       CAST(SUM(li.l_quantity) AS STRING) AS ans
FROM yogurt.silver.orders o
JOIN yogurt.silver.lineitem li ON li.l_orderkey = o.o_orderkey
WHERE o.o_custkey = 446044

UNION ALL
SELECT 'Q2_first_purchase_564787',
       DATE_FORMAT(MIN(TO_DATE(o_orderdate)), 'yyyy-MM-dd')
FROM yogurt.silver.orders
WHERE o_custkey = 564787

UNION ALL
SELECT 'Q3_total_orders_85909',
       CAST(COUNT(*) AS STRING)
FROM yogurt.silver.orders
WHERE o_custkey = 85909

UNION ALL
SELECT 'Q4_top_customer_nationality',
       n.n_name
FROM yogurt.silver.customer c
JOIN yogurt.silver.nation n ON c.c_nationkey = n.n_nationkey
JOIN (
  SELECT o.o_custkey
  FROM yogurt.silver.orders o
  GROUP BY o.o_custkey
  ORDER BY COUNT(*) DESC
  LIMIT 1
) m ON m.o_custkey = c.c_custkey;
